## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Current Description
0,0,Cape Town,ZA,-33.9258,18.4232,57.61,15,0,6.91,2021-07-26 02:22:57,clear sky
1,1,Hilo,US,19.7297,-155.0900,75.61,94,90,8.05,2021-07-26 02:22:57,mist
2,2,Pine Bluff,US,34.2284,-92.0032,84.13,70,20,0.00,2021-07-26 02:20:08,few clouds
3,3,Rikitea,PF,-23.1203,-134.9692,71.51,73,22,6.02,2021-07-26 02:22:58,few clouds
4,4,Lesozavodsk,RU,45.4794,133.4244,90.30,43,100,11.41,2021-07-26 02:22:58,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Current Description
0,0,Cape Town,ZA,-33.9258,18.4232,57.61,15,0,6.91,2021-07-26 02:22:57,clear sky
1,1,Hilo,US,19.7297,-155.0900,75.61,94,90,8.05,2021-07-26 02:22:57,mist
3,3,Rikitea,PF,-23.1203,-134.9692,71.51,73,22,6.02,2021-07-26 02:22:58,few clouds
6,6,Agadir,MA,30.4202,-9.5982,64.85,79,0,4.50,2021-07-26 02:23:00,clear sky
7,7,San Luis De La Paz,MX,21.3000,-100.5167,68.70,77,5,10.71,2021-07-26 02:23:00,clear sky
9,9,Khatanga,RU,71.9667,102.5000,59.74,59,70,1.99,2021-07-26 02:23:01,broken clouds
12,12,Nikolskoye,RU,59.7035,30.7861,63.81,62,89,4.34,2021-07-26 02:18:39,overcast clouds
13,13,San Cristobal,VE,7.7669,-72.2250,69.98,99,97,2.66,2021-07-26 02:23:02,overcast clouds
18,18,Reforma,MX,18.1000,-91.0167,77.05,88,91,3.76,2021-07-26 02:23:03,overcast clouds
19,19,Srednekolymsk,RU,67.4500,153.6833,57.38,39,63,15.37,2021-07-26 02:23:03,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Date                   0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Current Description
0,0,Cape Town,ZA,-33.9258,18.4232,57.61,15,0,6.91,2021-07-26 02:22:57,clear sky
1,1,Hilo,US,19.7297,-155.0900,75.61,94,90,8.05,2021-07-26 02:22:57,mist
3,3,Rikitea,PF,-23.1203,-134.9692,71.51,73,22,6.02,2021-07-26 02:22:58,few clouds
6,6,Agadir,MA,30.4202,-9.5982,64.85,79,0,4.50,2021-07-26 02:23:00,clear sky
7,7,San Luis De La Paz,MX,21.3000,-100.5167,68.70,77,5,10.71,2021-07-26 02:23:00,clear sky
...,...,...,...,...,...,...,...,...,...,...,...
665,665,Kosh-Agach,RU,50.0000,88.6667,63.73,57,8,2.21,2021-07-26 02:25:50,clear sky
666,666,San Giovanni In Fiore,IT,39.2633,16.6996,64.87,36,6,4.50,2021-07-26 02:25:50,clear sky
668,668,Somerset West,ZA,-34.0840,18.8211,57.58,39,100,6.42,2021-07-26 02:25:50,overcast clouds
669,669,Yar-Sale,RU,66.8333,70.8333,51.69,68,89,6.26,2021-07-26 02:25:51,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,57.61,clear sky,-33.9258,18.4232,
1,Hilo,US,75.61,mist,19.7297,-155.0900,
3,Rikitea,PF,71.51,few clouds,-23.1203,-134.9692,
6,Agadir,MA,64.85,clear sky,30.4202,-9.5982,
7,San Luis De La Paz,MX,68.70,clear sky,21.3000,-100.5167,
9,Khatanga,RU,59.74,broken clouds,71.9667,102.5000,
12,Nikolskoye,RU,63.81,overcast clouds,59.7035,30.7861,
13,San Cristobal,VE,69.98,overcast clouds,7.7669,-72.2250,
18,Reforma,MX,77.05,overcast clouds,18.1000,-91.0167,
19,Srednekolymsk,RU,57.38,broken clouds,67.4500,153.6833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,57.61,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
1,Hilo,US,75.61,mist,19.7297,-155.0900,Hilo Hawaiian Hotel
3,Rikitea,PF,71.51,few clouds,-23.1203,-134.9692,People ThankYou
6,Agadir,MA,64.85,clear sky,30.4202,-9.5982,Iberostar Founty Beach
7,San Luis De La Paz,MX,68.70,clear sky,21.3000,-100.5167,Hotel Real San Luis
...,...,...,...,...,...,...,...
665,Kosh-Agach,RU,63.73,clear sky,50.0000,88.6667,Fermerskiy Dom v Kosh-Agache
666,San Giovanni In Fiore,IT,64.87,clear sky,39.2633,16.6996,Casa Federico B&B
668,Somerset West,ZA,57.58,overcast clouds,-34.0840,18.8211,Lourens River Guesthouse
669,Yar-Sale,RU,51.69,overcast clouds,66.8333,70.8333,Yalemd


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]


# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))